# Folder Setup + Paths + DB

In [1]:
from pathlib import Path
import os

import pandas as pd
import matplotlib.pyplot as plt
from dotenv import load_dotenv
from sqlalchemy import create_engine, text

pd.set_option("display.max_columns", 200)
pd.set_option("display.width", 140)

# --- Resolve repo root no matter where notebook is opened ---
ROOT = Path.cwd()
while ROOT.name.lower() != "ecommerceproject" and not (ROOT / "analysis").exists():
    if ROOT.parent == ROOT:
        raise RuntimeError("Could not find repo root. Open the notebook from inside C:\\dev\\ecommerceproject")
    ROOT = ROOT.parent

OUT_TABLES = ROOT / "analysis" / "outputs" / "tables"
OUT_FIGS   = ROOT / "analysis" / "outputs" / "figures"
OUT_TABLES.mkdir(parents=True, exist_ok=True)
OUT_FIGS.mkdir(parents=True, exist_ok=True)

# --- Load env + connect ---
load_dotenv(ROOT / ".env")
db_url = os.getenv("DATABASE_URL")
if not db_url:
    raise RuntimeError("DATABASE_URL not found. Check C:\\dev\\ecommerceproject\\.env")

engine = create_engine(db_url)

# quick connectivity check
with engine.connect() as conn:
    conn.execute(text("SELECT 1"))
print("✅ Connected")
print("Tables dir:", OUT_TABLES)
print("Figures dir:", OUT_FIGS)

✅ Connected
Tables dir: C:\dev\ecommerceproject\analysis\outputs\tables
Figures dir: C:\dev\ecommerceproject\analysis\outputs\figures


# Pull the Returns dataset

In [2]:
sql = """
WITH bounds AS (
  SELECT MAX(created_at)::timestamptz AS max_ts
  FROM thelook_analytics.v_fact_order_items_enriched
),
base AS (
  SELECT
    order_item_id,
    order_id,
    user_id,
    product_id,
    inventory_item_id,
    created_at::timestamptz AS created_at,
    shipped_at::timestamptz AS shipped_at,
    delivered_at::timestamptz AS delivered_at,
    returned_at::timestamptz AS returned_at,
    sale_price::numeric AS sale_price,
    inventory_cost::numeric AS inventory_cost,
    (sale_price - inventory_cost)::numeric AS margin_proxy,
    category,
    department,
    brand,
    distribution_center_id,
    distribution_center_name,
    CASE WHEN returned_at IS NOT NULL THEN 1 ELSE 0 END AS is_returned,
    EXTRACT(EPOCH FROM (delivered_at::timestamptz - shipped_at::timestamptz))/86400.0 AS ship_to_deliver_days
  FROM thelook_analytics.v_fact_order_items_enriched
  WHERE created_at::timestamptz >= (SELECT max_ts - INTERVAL '12 months' FROM bounds)
)
SELECT *
FROM base;
"""

df = pd.read_sql_query(text(sql), engine)
df.shape, df.head(3)

((95717, 19),
    order_item_id  order_id  user_id  product_id  inventory_item_id                created_at                shipped_at  \
 0          97483     67087    53604       28341             263027 2023-04-03 01:39:12+00:00                       NaT   
 1         119813     82518    65843       28341             323357 2023-06-04 02:28:58+00:00                       NaT   
 2         170130    117171    93606       28341             459315 2023-06-23 14:06:13+00:00 2023-06-22 23:00:00+00:00   
 
                delivered_at returned_at  sale_price  inventory_cost  margin_proxy     category department     brand  \
 0                       NaT         NaT       24.44         10.2648       14.1752  Accessories        Men  Isotoner   
 1                       NaT         NaT       24.44         10.2648       14.1752  Accessories        Men  Isotoner   
 2 2023-06-23 01:31:00+00:00         NaT       24.44         10.2648       14.1752  Accessories        Men  Isotoner   
 
    distri

# Sanity checks + quick insights

In [3]:
kpis = {
    "rows": len(df),
    "unique_orders": df["order_id"].nunique(),
    "unique_users": df["user_id"].nunique(),
    "returned_items": int(df["is_returned"].sum()),
    "return_rate": float(df["is_returned"].mean()),
    "revenue": float(df["sale_price"].sum()),
    "cost_proxy": float(df["inventory_cost"].sum()),
    "margin_proxy": float(df["margin_proxy"].sum()),
    "margin_rate_proxy": float(df["margin_proxy"].sum() / df["sale_price"].sum()) if df["sale_price"].sum() else None,
}

kpis


{'rows': 95717,
 'unique_orders': 66019,
 'unique_users': 48277,
 'returned_items': 9721,
 'return_rate': 0.10155980651294963,
 'revenue': 5738761.640000001,
 'cost_proxy': 2759926.4706699997,
 'margin_proxy': 2978835.16933,
 'margin_rate_proxy': 0.5190728167845632}

### Quick insights (last 12 months window)
- Total revenue: **$X**, with proxy margin rate **Y%** (sale_price - inventory_cost).
- Return rate: **Z%** across **N** order items.
- Orders: **A** total, from **B** unique customers.


# Returns trend by month (return rate + revenue trend)

In [ ]:
from sqlalchemy import text
import pandas as pd
import matplotlib.pyplot as plt

sql = """
WITH bounds AS (
  SELECT MAX(created_at) AS max_ts
  FROM thelook_analytics.v_fact_order_items_enriched
),
base AS (
  SELECT *
  FROM thelook_analytics.v_fact_order_items_enriched
  WHERE created_at >= (SELECT max_ts - INTERVAL '12 months' FROM bounds)
)
SELECT
  date_trunc('month', created_at)::date AS month,
  COUNT(*) AS order_items,
  COUNT(*) FILTER (WHERE returned_at IS NOT NULL) AS returned_items,
  ROUND(
    (COUNT(*) FILTER (WHERE returned_at IS NOT NULL))::numeric / NULLIF(COUNT(*),0),
    4
  ) AS return_rate,
  SUM(sale_price) AS revenue,
  SUM(CASE WHEN returned_at IS NOT NULL THEN sale_price ELSE 0 END) AS returned_revenue
FROM base
GROUP BY 1
ORDER BY 1;
"""

with engine.connect() as conn:
    returns_trend = pd.read_sql_query(text(sql), conn)

display(returns_trend)

# Charts
fig = plt.figure()
plt.plot(returns_trend["month"], returns_trend["return_rate"])
plt.title("Return Rate (Last 12 Months)")
plt.xlabel("Month")
plt.ylabel("Return Rate")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

fig = plt.figure()
plt.plot(returns_trend["month"], returns_trend["revenue"])
plt.title("Revenue Trend (Last 12 Months)")
plt.xlabel("Month")
plt.ylabel("Revenue")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


# Worst categories/brands by return rate (and how much margin they burn)

In [ ]:
from sqlalchemy import text
import pandas as pd

sql = """
WITH bounds AS (
  SELECT MAX(created_at) AS max_ts
  FROM thelook_analytics.v_fact_order_items_enriched
),
base AS (
  SELECT *
  FROM thelook_analytics.v_fact_order_items_enriched
  WHERE created_at >= (SELECT max_ts - INTERVAL '12 months' FROM bounds)
)
SELECT
  category,
  COUNT(*) AS items,
  COUNT(*) FILTER (WHERE returned_at IS NOT NULL) AS returned_items,
  ROUND(
    (COUNT(*) FILTER (WHERE returned_at IS NOT NULL))::numeric / NULLIF(COUNT(*),0),
    4
  ) AS return_rate,
  SUM(sale_price) AS revenue,
  SUM(sale_price - inventory_cost) AS margin_proxy,
  SUM(CASE WHEN returned_at IS NOT NULL THEN (sale_price - inventory_cost) ELSE 0 END) AS margin_burn_proxy
FROM base
GROUP BY 1
HAVING COUNT(*) >= 300
ORDER BY margin_burn_proxy DESC, return_rate DESC
LIMIT 20;
"""

with engine.connect() as conn:
    worst_categories = pd.read_sql_query(text(sql), conn)

display(worst_categories)
